# Individual Income Tax Analysis


As a tax analyst who is also interested in data analysis, it is natually for me to be curious about various tax data. Since it is the time of the year to make estimate payment for individual tax, I am more interested in individual tax. I will analyze the individual income tax data for the past 5 years.

Here are some initial quesiton for exploration:

* How many people file tax returns each year?
* How many people use paid tax preparer?
* Relationship between salaries and AGI and total tax
* Which state pays the most tax?
* Which tax bracket has most people, and which tax bracket has the most tax?
* After the tax reform, what will the distribution look like?
* Will the individual income tax increase or decrease after the reform?
* There are a small group of people with higher tax rate than before. How many people are there and where do most of these people live?

## Dataset

The dataset can be found in [Kaggle](https://www.kaggle.com/irs/individual-income-tax-statistics/data) or [IRS](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi). It is saved separately as csv files for analysis. The detail information about column name and explanation can be found in "field_definitions.csv" file. Detail information can also be found in [IRS](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi). The original dataset is not consistent each year, so I will only use the following columns for this analysis. I have manipulate them to make it easiler to understand.

* state - 2 letter state abbreviation
* zipcode
* agi_class
* num_of_returns - number of returns
* agi - AGI
* total_wages - total amount of salaries & wages
* total_capital_gl - total amount of capital gain/loss
* total_tax - Total income tax amount
* prep - Number of returns using a Paid Preparer

AGI_Stub information shows below:

* 1 = \$1 under \$25,000 
* 2 = \$25,000 under \$50,000 
* 3 = \$50,000 under \$75,000 
* 4 = \$75,000 under \$100,000 
* 5 = \$100,000 under \$200,000 
* 6 = \$200,000 or more

## Getting Started

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [48]:
%run read_files.py

In [49]:
raw_data.head()

,state,zipcode,agi_class,num_of_returns,agi,total_wages,total_capital_gl,total_tax,prep,year
0,AK,99501,1.0,1459.0,1709,5588.0,209,116,730,2005
1,AK,99501,2.0,1713.0,29943,24052.0,180,1436,927,2005
2,AK,99501,3.0,1896.0,67861,57409.0,586,5896,905,2005
3,AK,99501,4.0,839.0,51347,36871.0,1687,6245,467,2005
4,AK,99501,5.0,417.0,35984,24282.0,981,5134,227,2005


In [61]:
raw_data.dtypes

state                object
zipcode               int64
agi_class           float64
num_of_returns      float64
agi                  object
total_wages         float64
total_capital_gl     object
total_tax            object
prep                 object
year                  int64
dtype: object

In [62]:
raw_data['agi_class'] = raw_data['agi_class'].astype('object')

In [67]:
raw_data[['agi', 'total_capital_gl', 'total_tax', 'prep']] = raw_data[
    ['agi', 'total_capital_gl', 'total_tax', 'prep']].apply(pd.to_numeric, errors='coerce')

In [68]:
raw_data.dtypes

state                object
zipcode               int64
agi_class            object
num_of_returns      float64
agi                 float64
total_wages         float64
total_capital_gl    float64
total_tax           float64
prep                float64
year                  int64
dtype: object

In [69]:
raw_data.shape

(1953802, 10)

In [71]:
raw_data.describe()

,zipcode,num_of_returns,agi,total_wages,total_capital_gl,total_tax,prep,year
count,1.953802e+06,1.931287e+06,1.922207e+06,1.928778e+06,1.892918e+06,1.872302e+06,1.910656e+06,1.953802e+06
mean,4.241869e+04,1.313526e+03,8.434421e+03,8.550801e+06,6.079098e+06,5.754828e+05,1.085569e+06,2.009570e+03
std,3.059228e+04,3.329159e+04,6.322080e+04,6.023600e+07,3.450416e+07,1.647389e+07,1.294756e+07,3.339095e+00
min,0.000000e+00,0.000000e+00,-4.407640e+05,-8.545497e+06,-1.298000e+03,-2.309389e+06,0.000000e+00,2.005000e+03
25%,1.471500e+04,5.000000e+01,5.100000e+01,3.625000e+03,5.050000e+02,1.000000e-04,1.200000e+02,2.006000e+03
50%,4.205800e+04,1.900000e+02,2.500000e+02,2.127500e+04,9.930000e+03,2.340000e+02,1.100000e+03,2.010000e+03
75%,6.743800e+04,8.200000e+02,1.350000e+03,1.980748e+05,1.510032e+05,2.863000e+03,1.415100e+04,2.013000e+03
max,9.999900e+04,1.557362e+07,1.791496e+07,1.516317e+10,5.177843e+09,7.993815e+09,3.872201e+09,2.015000e+03


In [72]:
tax_data = pd.pivot_table(raw_data, values='agi',
                         index=['year'], columns=['state'], aggfunc=np.sum)

In [73]:
tax_data

state,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
year,,,,,,,,,,,,,,,,,,,,,
2005,1.617382e+07,8.691169e+07,4.722142e+07,1.341480e+08,9.506890e+08,1.247819e+08,1.304361e+08,1.925834e+07,2.297680e+07,4.667008e+08,...,1.600203e+07,1.220252e+08,4.968350e+08,5.032337e+07,2.104606e+08,1.453034e+07,1.665800e+08,1.309919e+08,3.000661e+07,1.402025e+07
2006,1.738546e+07,9.374150e+07,4.982689e+07,1.453026e+08,1.016227e+09,1.370753e+08,1.402874e+08,2.090154e+07,2.385977e+07,4.987639e+08,...,1.731776e+07,1.316412e+08,5.532683e+08,5.648599e+07,2.250394e+08,1.544305e+07,1.833851e+08,1.391369e+08,3.145965e+07,3.262004e+07
2007,1.613020e+05,1.367944e+06,8.095150e+05,1.532714e+06,1.065315e+07,1.213146e+06,1.004356e+06,1.507230e+05,2.176110e+05,5.122105e+06,...,2.376500e+05,1.683673e+06,5.930587e+06,5.923270e+05,1.854114e+06,1.702060e+05,1.472523e+06,1.553871e+06,4.339100e+05,1.531670e+05
2008,1.548140e+05,1.282364e+06,7.462050e+05,1.457584e+06,1.030473e+07,1.165624e+06,9.604750e+05,1.467420e+05,2.061160e+05,4.827494e+06,...,2.123430e+05,1.553372e+06,5.698883e+06,5.730720e+05,1.760493e+06,1.558390e+05,1.403350e+06,1.482969e+06,3.783520e+05,1.447170e+05
2009,2.439420e+05,2.466978e+06,1.448414e+06,2.552708e+06,2.019837e+07,2.223482e+06,1.860740e+06,2.934980e+05,4.032760e+05,9.239214e+06,...,4.256280e+05,2.976406e+06,1.112039e+07,1.106894e+06,3.441630e+06,2.989200e+05,2.702042e+06,2.880014e+06,7.446060e+05,2.594200e+05
2010,2.472740e+05,2.451302e+06,1.451586e+06,2.652548e+06,2.039359e+07,2.252570e+06,1.868900e+06,2.946540e+05,4.086540e+05,9.377010e+06,...,4.287520e+05,2.986936e+06,1.117879e+07,1.105206e+06,3.449054e+06,2.996180e+05,2.721008e+06,2.904918e+06,7.484361e+05,2.622500e+05
2011,3.164160e+05,2.466524e+06,1.494820e+06,2.860666e+06,2.102571e+07,2.354500e+06,1.894648e+06,3.006620e+05,4.121300e+05,9.672370e+06,...,4.462160e+05,3.011242e+06,1.160860e+07,1.158430e+06,3.510302e+06,3.102280e+05,2.778868e+06,2.943820e+06,7.894921e+05,2.968020e+05
2012,3.080700e+05,2.441700e+06,1.474630e+06,2.821590e+06,2.114538e+07,2.355670e+06,1.884930e+06,3.029800e+05,4.063000e+05,9.745860e+06,...,4.500400e+05,2.973610e+06,1.174243e+07,1.176560e+06,3.463040e+06,3.063200e+05,2.759930e+06,2.935130e+06,7.668300e+05,2.940100e+05
2013,3.009400e+05,2.391570e+06,1.444980e+06,2.811290e+06,2.139720e+07,2.386360e+06,1.879120e+06,3.030100e+05,4.086700e+05,9.821740e+06,...,4.460600e+05,2.919350e+06,1.179966e+07,1.192210e+06,3.405660e+06,3.038500e+05,2.751270e+06,2.917590e+06,7.510900e+05,2.877500e+05


In [59]:
tax_07

,state,zipcode,agi_class,num_of_returns,agi,total_wages,total_capital_gl,total_tax,prep,year
0,AL,35001,1,116,.,14867,.,.,.,2007
1,AL,35004,1,916,364,2810190,2536200,54545,14885,2007
2,AL,35004,2,913,499,16042711,12632831,84134,432577,2007
3,AL,35004,3,1317,719,48447676,42594042,281953,2944383,2007
4,AL,35004,4,904,519,55742354,49181538,347714,4525205,2007
5,AL,35004,5,531,273,45920811,41685154,217739,4266948,2007
6,AL,35004,6,351,214,44136430,35940143,1034942,5667499,2007
7,AL,35004,7,41,37,21282039,10219196,1989084,5005174,2007
8,AL,35005,1,1052,405,3285751,2705002,-590,12127,2007
9,AL,35005,2,981,664,16915480,13063815,49077,350174,2007


In [47]:
tax_06['state'].str.upper()

0         AK
1         AK
2         AK
3         AK
4         AK
5         AK
6         AK
7         AK
8         AK
9         AK
10        AK
11        AK
12        AK
13        AK
14        AK
15        AK
16        AK
17        AK
18        AK
19        AK
20        AK
21        AK
22        AK
23        AK
24        AK
25        AK
26        AK
27        AK
28        AK
29        AK
          ..
269573    WY
269574    WY
269575    WY
269576    WY
269577    WY
269578    WY
269579    WY
269580    WY
269581    WY
269582    WY
269583    WY
269584    WY
269585    WY
269586    WY
269587    WY
269588    WY
269589    WY
269590    WY
269591    WY
269592    WY
269593    WY
269594    WY
269595    WY
269596    WY
269597    WY
269598    WY
269599    WY
269600    WY
269601    WY
269602    WY
Name: state, Length: 269603, dtype: object

In [46]:
tax_07.head()

,state,zipcode,agi_class,num_of_returns,agi,total_wages,total_capital_gl,total_tax,prep,year
0,AL,35001,1,116,.,14867,.,.,.,2007
1,AL,35004,1,916,364,2810190,2536200,54545,14885,2007
2,AL,35004,2,913,499,16042711,12632831,84134,432577,2007
3,AL,35004,3,1317,719,48447676,42594042,281953,2944383,2007
4,AL,35004,4,904,519,55742354,49181538,347714,4525205,2007


## Explore the Dataset

In [27]:
def create_df(columns):
    columns = pd.pivot_table(raw_data, values=columns, index='year', columns='state', aggfunc=np.sum)
    return columns

In [28]:
for col in raw_data.columns:
    col = create_df(col)

ValueError: Grouper for 'state' not 1-dimensional